In [1]:
import pandas as pd
import numpy as np
import copy
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

Read the data:
File train.csv contains anotated data.
File test.csv contains data on which predictions have to be made and submited for the competition.

In [2]:
df = pd.read_csv("files_task1/train.csv")
df_test = pd.read_csv("files_task1/test.csv")
df_test["target_flag"] = 0
df_test["target_sum"] = 0
# df_comb is all the data in one dataframe
df_comb = pd.concat([df,df_test])

Chek for the same IDs in the two data sets

In [30]:
pd.Index(df["cl_id"]).intersection(pd.Index(df_test["cl_id"]))
# The intersection is empty

Int64Index([], dtype='int64', name='cl_id')

In [4]:
df.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [26]:
df.shape

(490513, 10)

Find how many missing values are in eavry column

In [6]:
(df_comb["MCC"].isnull().sum(),
df_comb["channel_type"].isnull().sum(),
df_comb["TRDATETIME"].isnull().sum(),
df_comb["currency"].isnull().sum(),
df_comb["amount"].isnull().sum(),
df_comb["target_flag"].isnull().sum(),
df_comb["target_sum"].isnull().sum())

(0, 5064, 0, 0, 0, 0, 0)

Information about the "trx_category" column

In [7]:
print((df["trx_category"].unique() != df_comb["trx_category"].unique()).sum())
df["trx_category"].unique()

0


array(['POS', 'DEPOSIT', 'C2C_OUT', 'WD_ATM_ROS', 'BACK_TRX',
       'WD_ATM_PARTNER', 'WD_ATM_OTHER', 'C2C_IN', 'CAT', 'CASH_ADV'],
      dtype=object)

Chanal type is a colmun regrading the chanel used to give offer to the user

In [8]:
df["channel_type"].value_counts()

type1    299247
type2    175013
type5      5587
type4      4476
type3      3280
Name: channel_type, dtype: int64

Convert the columns containing information about the time of the transaction to suitable type and format

In [4]:
df = df.fillna("unknown")
df["PERIOD"] = pd.to_datetime(df["PERIOD"])
df_comb = df_comb.fillna("unknown")
df_comb["PERIOD"] = pd.to_datetime(df_comb["PERIOD"])
df["TRDATETIME"] = df["TRDATETIME"].apply(lambda x: float(x[5:7])+float(x[8:10])/60)
df_comb["TRDATETIME"] = df_comb["TRDATETIME"].apply(lambda x: float(x[5:7])+float(x[8:10])/60)

Dictonary containing ruff estamates for code of currency - average exchange rate for 2017(RUB)

In [5]:
conv_dict_cur_to_rub = {32:3.5, 36:44, 44:60, 48:155, 51:0.12, 124:44, 144:0.38, 156:8.5, 170:0.02, 188:0.1, 191:8.6, 203:2.5, 208:8.9, 214:1.3, 344:7.5, 348:0.21, 352:0.53, 356:0.9, 360:0.0045, 376:16, 392:0.52, 398:0.18, 404:0.6, 410:0.0515, 417:0.95, 446:7.58, 458:14.5, 462:3.95, 480:1.7, 484:3, 496:0.025, 498:3.3, 504:6.4, 516:4.5, 524:0.56, 554:42, 578:7, 604:18.7, 608:1.15, 634:16, 643:1, 690:4.5, 702:42, 704:0.0027, 710:4.5, 752:6.8, 756:59.9, 764:1.73, 784:15.9, 788:23.7, 800:0.017, 810:1, 826:75, 840:58, 901:1.91, 933:30.6, 941:4.5, 944:35.9, 946:15.5, 949:16, 975:36.9, 977:36.9, 978:72.1, 980:2.35, 981:25, 985:16.8, 986:16.8}

Engineer features for the users.
Two diffrent ways to represent the data: 
1. User behaviour before and during the period of using the special service. 
There is no data for all useres prior to the special service.
2. User behaviour for the whole period. 


In [6]:
# user IDs
ids = []
# Time diffrence between the first and last transaction
time_deltas = []

# Columns to predict
# target_flags - Will the user continue to use the service
# target_sums - Flow of transaction from POS TERMINAL
target_flags = []
target_sums = []


# Total number of transactions
num_transactions = []
num_transactions_before = []
num_transactions_during = []

# How many types of currencies the user have used
num_currancy = []
num_currancy_before = []
num_currancy_during = []

# In how many diffrent types of places the user have spended money
mcc_var = []
mcc_var_before = []
mcc_var_during = []

# How many diffrent types of transcation the user have executed
trans_var = []
trans_var_before = []
trans_var_during = []

# Mean time of the day for the transcations
trd_time_mean = []
trd_time_mean_before = []
trd_time_mean_during = []

# STD of the mean time of the day for the transcations
trd_time_std = []
trd_time_std_before = []
trd_time_std_during = []

# Total flow of money(RUB)
total_rub_cur = []
total_rub_cur_before = []
total_rub_cur_during = []

# Mean flow of money(RUB)
mean_rub_cur = []
mean_rub_cur_before = []
mean_rub_cur_during = []

# Output flow(RUB)
total_output = []
total_output_before = []
total_output_during = []

# Input flow(RUB)
total_input = []
total_input_during = []
total_input_before = []

# Output flow/Input flow
total_net = []
total_net_before = []
total_net_during = []

num_POS_transactions_during = []
num_POS_transactions_before = []
amt_POS_transactions_during = []
amt_POS_transactions_before = []


# Chanel used to give the offer to user. The chanels except 1 and 2 are not offten used.
chanels_dict= {'type2':[],'type1':[]}
chanels_mapping = {'type2':2, 'type1': 1}

# Groups of simmilar types of transactions
input_flow_types = ["C2C_IN","DEPOSIT"]
WD_ATM = ['WD_ATM_ROS' ,'WD_ATM_PARTNER','WD_ATM_OTHER','CASH_ADV']
C2C = ['C2C_IN','C2C_OUT']
POS_CAT = ["POS", "CAT"]

# Dict. that will contain the Amount of money for the transaction type/ Total flow of money
total_transactions_dict = {"WD_ATM":[],"DEPOSIT":[],"C2C":[],"BACK_TRX":[],"POS_CAT":[]}
emtpy_transactions_dict =  {"WD_ATM":0,"DEPOSIT":0,"C2C":0,"BACK_TRX":0,"POS_CAT":0}
total_transactions_dict_before = copy.deepcopy(total_transactions_dict)
total_transactions_dict_during = copy.deepcopy(total_transactions_dict)


before_features = [num_transactions_before, num_currancy_before ,mcc_var_before,
                   trans_var_before, trd_time_mean_before, trd_time_std_before, 
                   total_rub_cur_before, mean_rub_cur_before, total_output_before,
                   total_input_before, total_net_before]

during_features = [num_transactions_during, num_currancy_during, mcc_var_during, 
                   trans_var_during, trd_time_mean_during, trd_time_std_during, 
                   total_rub_cur_during, mean_rub_cur_during, total_output_during,
                   total_input_during, total_net_during]

# Iterate over all users 
for i_d in df_comb["cl_id"].unique():
    ids.append(i_d)  
    mini_df = df_comb[df_comb["cl_id"]== i_d]
    min_time = mini_df["PERIOD"].min()
    max_time = mini_df["PERIOD"].max()
    time_delta = abs(int((max_time - min_time).days))
    time_deltas.append(time_delta)    
    trans_num = mini_df.shape[0]
    num_transactions.append(trans_num)
    
    num_currancy.append(len(list(mini_df["currency"].unique())))
    mcc_var.append(len(list(mini_df["MCC"].unique())))
    trans_var.append(len(list(mini_df["trx_category"].unique())))
    trd_time_mean.append(mini_df["TRDATETIME"].mean())
    trd_time_std.append(mini_df["TRDATETIME"].std())
    
    target_sums.append(mini_df["target_sum"].mean())
    target_flags.append(mini_df["target_flag"].mean())
    total_user_rub_cur = 0
    user_output = 0
    user_input = 0
    for i in range(trans_num):
        row = mini_df.iloc[i,:]
        type_curr = row["currency"]       
        transact_amount_user = conv_dict_cur_to_rub[type_curr]*row["amount"]
        total_user_rub_cur =+ transact_amount_user
        if row["trx_category"] in input_flow_types:
            user_input += transact_amount_user
        else:
            user_output += transact_amount_user
    ch = mini_df["channel_type"].unique()[0]
    if ch in chanels_dict:
        chanels_dict[ch].append(chanels_mapping[ch])
    for key in chanels_dict:
        if key != ch:
            chanels_dict[key].append(0)
    
    total_rub_cur.append(total_user_rub_cur)
    mean_rub_cur.append(total_user_rub_cur/trans_num)
    
    total_output.append(user_output)
    total_input.append(user_input)
    if user_input == 0:
        total_net.append(1)
    else:
        total_net.append(user_output/user_input)
    

    user_transactions_dict = copy.deepcopy(emtpy_transactions_dict)
    user_amount_of_total_transactions_dict = copy.deepcopy(emtpy_transactions_dict)

    for i in range(trans_num):
        row = mini_df.iloc[i,:]
        trx = row["trx_category"]
        type_curr = row["currency"]
        amount_rub = conv_dict_cur_to_rub[type_curr]*row["amount"]
        if trx in WD_ATM:
            user_amount_of_total_transactions_dict["WD_ATM"] += amount_rub
        elif trx in C2C:
             user_amount_of_total_transactions_dict["C2C"] += amount_rub
        elif trx in POS_CAT:
            user_amount_of_total_transactions_dict["POS_CAT"] += amount_rub
                
        else:
            user_amount_of_total_transactions_dict[trx] += amount_rub
                
                
                
                
            
    for t in user_amount_of_total_transactions_dict:
        total_transactions_dict[t].append(user_amount_of_total_transactions_dict[t]/total_user_rub_cur)

for i_d in df_comb["cl_id"].unique():
    client_df = df_comb[df_comb["cl_id"]==i_d]
    max_time = client_df["PERIOD"].max()
    start_time = max_time - timedelta(days=90)
    mini_df_before = client_df[client_df["PERIOD"] < start_time]
    mini_df_during = client_df[client_df["PERIOD"] >= start_time]
    min_time = client_df["PERIOD"].min()
    
    num_days = (abs(int((max_time - min_time).days))-90)
    k = 90/num_days
    
    num_POS_transactions_during.append((mini_df_during["trx_category"]=="POS").sum()*k)
    num_POS_transactions_before.append((mini_df_before["trx_category"]=="POS").sum()*k)
    trans_num = mini_df_before.shape[0]
    if trans_num == 0:
        num_POS_transactions_before.append(0)
        amt_POS_transactions_before.append(0)
        for f in before_features:
            f.append(0)
        for key in total_transactions_dict_before:
            total_transactions_dict_before[key].append(0)
    else:
        num_transactions_before.append(trans_num*k)

        num_currancy_before.append(len(list(mini_df_before["currency"].unique())))
        mcc_var_before.append(len(list(mini_df_before["MCC"].unique())))
        trans_var_before.append(len(list(mini_df_before["trx_category"].unique())))
        trd_time_mean_before.append(mini_df_before["TRDATETIME"].mean())
        trd_time_std_before.append(mini_df_before["TRDATETIME"].std())
        total_user_rub_cur = 0
        user_output = 0
        user_input = 0
        for i in range(trans_num):
            row = mini_df_before.iloc[i,:]
            type_curr = row["currency"]       
            transact_amount_user = conv_dict_cur_to_rub[type_curr]*row["amount"]
            total_user_rub_cur =+ transact_amount_user
            if row["trx_category"] in input_flow_types:
                user_input += transact_amount_user
            else:
                user_output += transact_amount_user

        total_rub_cur_before.append(total_user_rub_cur*k)
        mean_rub_cur_before.append(k*total_user_rub_cur/trans_num)

        total_output_before.append(k*user_output)
        total_input_before.append(k*user_input)
        if user_input == 0:
            total_net_before.append(1)
        else:
            total_net_before.append(user_output/user_input)

        user_amount_of_total_transactions_dict = copy.deepcopy(emtpy_transactions_dict)

        for i in range(trans_num):
            row = mini_df_before.iloc[i,:]
            trx = row["trx_category"]
            type_curr = row["currency"]
            amount_rub = conv_dict_cur_to_rub[type_curr]*row["amount"]*k
            if trx in WD_ATM:
                user_amount_of_total_transactions_dict["WD_ATM"] += amount_rub
            elif trx in C2C:
                 user_amount_of_total_transactions_dict["C2C"] += amount_rub
            elif trx in POS_CAT:
                user_amount_of_total_transactions_dict["POS_CAT"] += amount_rub
            else:
                user_amount_of_total_transactions_dict[trx] += amount_rub



        amt_POS_transactions_before.append(user_amount_of_total_transactions_dict["POS_CAT"]*k)
        for t in user_amount_of_total_transactions_dict:
            total_transactions_dict_before[t].append(user_amount_of_total_transactions_dict[t]*k)  
            
    trans_num = mini_df_during.shape[0]
    if trans_num == 0:
        num_POS_transactions_during.append(0)
        amt_POS_transactions_during.append(0)
        for f in during_features:
            f.append(0)
        for key in total_transactions_dict_during:
            total_transactions_dict_during[key].append(0)
    else:
        num_transactions_during.append(trans_num)

        num_currancy_during.append(len(list(mini_df_during["currency"].unique())))
        mcc_var_during.append(len(list(mini_df_during["MCC"].unique())))
        trans_var_during.append(len(list(mini_df_during["trx_category"].unique())))
        trd_time_mean_during.append(mini_df_during["TRDATETIME"].mean())
        trd_time_std_during.append(mini_df_during["TRDATETIME"].std())
        total_user_rub_cur = 0
        user_output = 0
        user_input = 0
        for i in range(trans_num):
            row = mini_df_during.iloc[i,:]
            type_curr = row["currency"]       
            transact_amount_user = conv_dict_cur_to_rub[type_curr]*row["amount"]
            total_user_rub_cur =+ transact_amount_user
            if row["trx_category"] in input_flow_types:
                user_input += transact_amount_user
            else:
                user_output += transact_amount_user

        total_rub_cur_during.append(total_user_rub_cur)
        mean_rub_cur_during.append(total_user_rub_cur/trans_num)

        total_output_during.append(user_output)
        total_input_during.append(user_input)
        if user_input == 0:
            total_net_during.append(1)
        else:
            total_net_during.append(user_output/user_input)

        user_amount_of_total_transactions_dict = copy.deepcopy(emtpy_transactions_dict)

        for i in range(trans_num):
            row = mini_df_during.iloc[i,:]
            trx = row["trx_category"]
            type_curr = row["currency"]
            amount_rub = conv_dict_cur_to_rub[type_curr]*row["amount"]
            if trx in WD_ATM:
                user_amount_of_total_transactions_dict["WD_ATM"] += amount_rub
            elif trx in C2C:
                 user_amount_of_total_transactions_dict["C2C"] += amount_rub
            elif trx in POS_CAT:
                user_amount_of_total_transactions_dict["POS_CAT"] += amount_rub
            else:
                user_amount_of_total_transactions_dict[trx] += amount_rub

        amt_POS_transactions_during.append(user_amount_of_total_transactions_dict["POS_CAT"])
        for t in user_amount_of_total_transactions_dict:
            total_transactions_dict_during[t].append(user_amount_of_total_transactions_dict[t]/total_user_rub_cur)


Uncomment for representation 2, however the 1st type gave better scores

In [ ]:
"""
named_feats = ["num_transactions", "num_currancy","mcc_var",
                   "trans_var", "trd_time_mean", "trd_time_std", 
                   "total_rub_cur", "mean_rub_cur", "total_output",
                   "total_input", "total_net"]

named_feats.append("time_deltas")
features = [num_transactions,num_currancy,mcc_var,trans_var, trd_time_mean,trd_time_std,
            total_rub_cur, mean_rub_cur, total_output, total_input, total_net, time_deltas]
dict_features = [total_transactions_dict,chanels_dict]
for dict_f in dict_features:
    i = 1
    for key in dict_f:
        i += 1
        named_feats.append(str(key) + "_" + str(i))
        final_features.append(dict_f[key])
"""

Before/During representation

In [7]:
named_feats = ["num_transactions", "num_currancy","mcc_var",
                   "trans_var", "trd_time_mean", "trd_time_std", 
                   "total_rub_cur", "mean_rub_cur", "total_output",
                   "total_input", "total_net"]

features =  before_features + during_features + [time_deltas]
before_named_feats = [x + "_before" for x in named_feats]
during_named_feats = [x + "_during" for x in named_feats]
named_feats = before_named_feats + during_named_feats + ["time_deltas"]
dict_features = [(total_transactions_dict_during, "during"), 
                 (total_transactions_dict_before,  "before"),
                 (chanels_dict, "chanel")]
final_features = list(features)
for dictt in dict_features:
    dict_f ,f_name= dictt
    i = 1
    for key in dict_f:
        i += 1
        named_feats.append(str(key) + "_" + f_name)
        final_features.append(dict_f[key])

Convert data to numpy array with dim. number samples, number attributes

In [8]:
X = np.array(final_features).T
X.shape

(10217, 35)

Fill NaN Values(They come from users who have made only one transaction , which leads to time std of NaN)

In [9]:
df_X = pd.DataFrame(X)
df_X = df_X.fillna(0)

Save the data for clustering

In [ ]:
df_X.to_pickle("df_task_3")

Find the location of the last example from the anotated set of examples. And separate the data to: data for submission, train and test sets.

In [10]:
number_of_anotated_users = df["cl_id"].unique().shape[0]
df_submisson = df_X.iloc[number_of_anotated_users:, :]
df_rest = df_X.iloc[:number_of_anotated_users, :]

df_submisson.to_pickle("df_t1_for_t2")
df_rest["flag"] = np.array(target_flags[:number_of_anotated_users])
df_rest["sums"] = np.array(target_sums[:number_of_anotated_users])

#shuffle the data
df_rest = shuffle(df_rest)
train_treshold = int(0.65*number_of_anotated_users)

df_train = df_rest.iloc[:train_treshold, :]
df_test = df_rest.iloc[train_treshold:, :]

C:\Users\EMO\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\EMO\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [11]:
df_rest[(df_rest["flag"]==0) & (df_rest["sums"]>0)].shape
# Sanity chek : 
#There are no usesrs who have stoped useing the service but still have registerd sums in the column target sums

(0, 37)

In [50]:
df_rest.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,flag,sums
639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,54.511729,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,57233.83
1978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.790321,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00
1631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,438.638485,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,210498.92
1085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,878.807500,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,91242.09


Distribution of classes:

In [15]:
print("% positive Class:",sum(target_flags[:number_of_anotated_users])/number_of_anotated_users)
print("Number of anotated users: ",df_rest.shape[0])

% positive Class: 0.5534
Number of anotated users:  5000


Scale the data to have 0 mean and unit variance.
X_train, X_sub, X_test are gona be the arrays used for task 1(binary classification)

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(df_train.iloc[:,:-2])
X_test = scaler.transform(df_test.iloc[:,:-2])
X_sub = scaler.transform(df_submisson)

Save the data for the  task 1, prior to this add the column to be predicted for each task

In [17]:
X_train_predict_flags = np.hstack([X_train, np.array(df_train["flag"].values.tolist()).reshape(train_treshold,1)])
dim_t = int(number_of_anotated_users-train_treshold)
X_test_predict_flags = np.hstack([X_test, np.array(df_test["flag"].values.tolist()).reshape(dim_t,1)])

In [18]:
np.save("X_sub.npy",X_sub)
np.save("X_train_flags.npy",X_train_predict_flags)
np.save("X_test_flags.npy",X_test_predict_flags)

Greate features for task 2(prediction of Transaction from POS)

In [19]:
def task_2_features(data_frame, named_feats, num_POS_transactions_during,num_POS_transactions_before, amt_POS_transactions_during, amt_POS_transactions_before): 
    data = data_frame.copy()
    n_trx_b = data.iloc[:,named_feats.index('num_transactions_before')]
    net_b = data.iloc[:,named_feats.index('total_net_before')]
    relative_pos_b = data.iloc[:,named_feats.index('POS_CAT_before')]
    
    Total_amount_from_POS_b = net_b*relative_pos_b
    n_trx_d = data.iloc[:,named_feats.index('num_transactions_during')]
    net_d = data.iloc[:,named_feats.index('total_net_during')]
    relative_pos_d = data.iloc[:,named_feats.index('POS_CAT_before')]
    Total_amount_from_POS_d = net_d*relative_pos_d
    
    result_df = pd.DataFrame(n_trx_b).rename(index=str, columns={0: "num_trx_before"})
    result_df["num_trx_during"] = n_trx_d.values.tolist()
    result_df["POS_CAT_before"] = Total_amount_from_POS_b.values.tolist()   
    result_df["POS_CAT_during"] = Total_amount_from_POS_d.values.tolist()
    result_df["POS_num_during"] = num_POS_transactions_during[:5000]
    result_df["POS_num_before"] = num_POS_transactions_before[:5000]
    result_df["POS_amt_before"] = amt_POS_transactions_before[:5000]
    result_df["POS_amt_during"] = amt_POS_transactions_during[:5000]
    result_df["sums"] = data["sums"].values.tolist()
    result_df["flags"] = data["flag"].values.tolist()
    return result_df 
    
df_task_2 = task_2_features(df_rest, named_feats,num_POS_transactions_during, num_POS_transactions_before,amt_POS_transactions_during, amt_POS_transactions_before)

In [ ]:
def task_sub_features(data_frame, named_feats, num_POS_transactions_during,num_POS_transactions_before, amt_POS_transactions_during, amt_POS_transactions_before): 
    data = data_frame.copy()
    n_trx_b = data.iloc[:,named_feats.index('num_transactions_before')]
    net_b = data.iloc[:,named_feats.index('total_net_before')]
    relative_pos_b = data.iloc[:,named_feats.index('POS_CAT_before')]
    
    Total_amount_from_POS_b = net_b*relative_pos_b
    n_trx_d = data.iloc[:,named_feats.index('num_transactions_during')]
    net_d = data.iloc[:,named_feats.index('total_net_during')]
    relative_pos_d = data.iloc[:,named_feats.index('POS_CAT_before')]
    Total_amount_from_POS_d = net_d*relative_pos_d
    
    result_df = pd.DataFrame(n_trx_b).rename(index=str, columns={0: "num_trx_before"})
    result_df["num_trx_during"] = n_trx_d.values.tolist()
    result_df["POS_CAT_before"] = Total_amount_from_POS_b.values.tolist()   
    result_df["POS_CAT_during"] = Total_amount_from_POS_d.values.tolist()
    result_df["POS_num_during"] = num_POS_transactions_during[5000:]
    result_df["POS_num_before"] = num_POS_transactions_before[5000:]
    result_df["POS_amt_before"] = amt_POS_transactions_before[5000:]
    result_df["POS_amt_during"] = amt_POS_transactions_during[5000:]

    return result_df 
    
df_task_2_sub = task_sub_features(df_submisson, named_feats,num_POS_transactions_during, num_POS_transactions_before,amt_POS_transactions_during, amt_POS_transactions_before)

In [20]:
df_task_2.head()

,num_trx_before,num_trx_during,POS_CAT_before,POS_CAT_during,POS_num_during,POS_num_before,POS_amt_before,POS_amt_during,sums,flags
3668,4.719101,21.0,194.461204,267.162233,-3.068182,-0.0,0.0,7821.000,0.00,0.0
4861,18.539326,110.0,9697.731373,9744.124906,-104.482759,0.0,0.0,185495.141,3247.50,1.0
1189,15.168539,78.0,12216.881058,12384.288600,-114.827586,-0.0,0.0,301976.281,170632.46,1.0
550,0.000000,49.0,0.000000,0.000000,-29.659091,0.0,0.0,12365.090,78148.40,1.0
1497,0.000000,129.0,0.000000,0.000000,-382.500000,-0.0,0.0,238651.090,0.00,0.0


Save the data for task 2 

In [23]:
df_task_2.to_pickle("df_task_2")
df_task_2_sub.to_pickle("df_task_2_sub")